In [46]:
# Name: Kathiriya Ranjit (R00183586)
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from textblob import Word
import re

nltk.download('stopwords')
stop = stopwords.words('english')


from pytorch_pretrained_bert import BertTokenizer,BertForMaskedLM
import torch
import pandas as pd
import math

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ranjitsmac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1]:
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

    1.    Load Dataset:

    a.    Load dataset:
        i.    In subtaskA_data_all.csv, contain 10,000 entries with column name id,falsesent,optionA, optionB, and OptionC.
        ii.    In subtaskA_answers_all.csv, contain 10,000 entries with column name id and sent0 of the sentence, which the option is true either in A,B,C.
        
        - same with test data from github.
        

In [30]:
# TRAIN DATA

df_all = pd.read_csv('./TrainingData/subtaskB_data_all.csv')

df_data = pd.read_csv('./TrainingData/subtaskB_answers_all.csv',header='infer',names=['id','target'])

df = pd.merge(df_all,df_data, on='id')

df['OptionC'].replace('', np.nan, inplace=True)

df.dropna(subset=['OptionC'],inplace=True)

df['new_target'] = df["target"].apply(lambda x: 0 if x=='A' else (1 if x=='B' else 2))

In [31]:
# TEST DATA

df_train_all = pd.read_csv('./TrialData/taskB_trial_data.csv')
df_train_data = pd.read_csv('./TrialData/taskB_trial_answer.csv',header='infer',names=['id','target'])

df_train = pd.merge(df_train_all,df_train_data, on='id')

df_train['OptionC'].replace('', np.nan, inplace=True)

df_train.dropna(subset=['OptionC'],inplace=True)

df_train['new_target'] = df_train["target"].apply(lambda x: 0 if x=='A' else (1 if x=='B' else 2))

# Cleaning 

    - In bert score the cleaning process will be not there because if we clean the data then bert score will be less. the main reason for this the sentence is not formed in proper way because of cleaning.

# BERT SCORE OF ALL DATAS

    - Bert score for all rows of FalseSent,OptionA,OptionB,and OptionC columns and store it into new columns named ScoreFalse,scoreA,scoreB, and scoreC

In [34]:
# initialize bert 
bertMaskedLM = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [35]:
# get score function for generating score.
def get_score(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions=bertMaskedLM(tensor_input)
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(predictions.squeeze(),tensor_input.squeeze()).data 
    return math.exp(loss)

In [36]:
df.head()

,id,FalseSent,OptionA,OptionB,OptionC,target,new_target
0,0,He poured orange juice on his cereal.,Orange juice is usually bright orange.,Orange juice doesn't taste good on cereal.,Orange juice is sticky if you spill it on the ...,B,1
1,1,He drinks apple.,Apple juice are very tasty and milk too,Apple can not be drunk,Apple cannot eat a human,B,1
2,2,"Jeff ran 100,000 miles today","100,000 miles is way to long for one person to...","Jeff is a four letter name and 100,000 has six...","100,000 miles is longer than 100,000 km.",A,0
3,3,I sting a mosquito,A human is a mammal,A human is omnivorous,A human has not stings,C,2
4,4,A giraffe is a person.,Giraffes can drink water from a lake.,A giraffe is not a human being.,.Giraffes usually eat leaves.,B,1


In [37]:
# optaning BERT score for all train colums
df['ScoreFalse'] = df['FalseSent'].apply(lambda x: get_score(x))

df['scoreA'] = df['OptionA'].apply(lambda x: get_score(x))

df['scoreB'] = df['OptionB'].apply(lambda x: get_score(x))

df['scoreC'] = df['OptionC'].apply(lambda x: get_score(x))

In [41]:
df.head()

,id,FalseSent,OptionA,OptionB,OptionC,target,new_target,ScoreFalse,scoreA,scoreB,scoreC
0,0,He poured orange juice on his cereal.,Orange juice is usually bright orange.,Orange juice doesn't taste good on cereal.,Orange juice is sticky if you spill it on the ...,B,1,174.785154,23.896005,18.129260,11.662107
1,1,He drinks apple.,Apple juice are very tasty and milk too,Apple can not be drunk,Apple cannot eat a human,B,1,261.141406,45.627990,70.528422,129.325482
2,2,"Jeff ran 100,000 miles today","100,000 miles is way to long for one person to...","Jeff is a four letter name and 100,000 has six...","100,000 miles is longer than 100,000 km.",A,0,36.537167,2.225370,8.375058,4.414386
3,3,I sting a mosquito,A human is a mammal,A human is omnivorous,A human has not stings,C,2,341.935915,30.989992,118.113837,48.933158
4,4,A giraffe is a person.,Giraffes can drink water from a lake.,A giraffe is not a human being.,.Giraffes usually eat leaves.,B,1,19.015774,35.124041,21.117710,3.898473


In [42]:
# optaning BERT score for all test colums
df_train['ScoreFalse'] = df_train['FalseSent'].apply(lambda x: get_score(x))

df_train['scoreA'] = df_train['OptionA'].apply(lambda x: get_score(x))

df_train['scoreB'] = df_train['OptionB'].apply(lambda x: get_score(x))

df_train['scoreC'] = df_train['OptionC'].apply(lambda x: get_score(x))

In [43]:
df_train.head()

,id,FalseSent,OptionA,OptionB,OptionC,target,new_target,ScoreFalse,scoreA,scoreB,scoreC
0,1,he put an elephant into the fridge,an elephant is much bigger than a fridge,elephants are usually gray while fridges are u...,an elephant cannot eat a fridge,A,0,10.064267,7.465191,59.716604,162.638716
1,2,my sister eats a stone after breakfast every day,stone is usually in round-shapes,no one eats stones,stone is too large for a girl's mouth,B,1,12.306699,26.744377,518.012578,5.249769
2,3,money can be used for buying stars,no one can get stars and sell them now,stars can be only seen in cloudless dark night,stars are too expensive for normal people,A,0,64.401493,11.673301,25.306239,124.302623
3,4,USA is located in the northeastern part of New...,New York is not the capital of USA,USA is a nation while New York is its ally,USA is a nation while New York is its city,C,2,6.695859,10.432017,4.280972,13.351408
4,5,a man can better see stars and the moon in day...,the sun is much brighter than the moon and sta...,the moon is much closer to the earth than the sun,the sun moves faster than the moon and stars,A,0,6.063322,4.019972,5.183219,24.241701


# Train test Split

     3.    Train test split 

            a.    For further evaluation, the dataset present in the test folder is taken and evaluated using three classification model.
            
    4. Apply 3 Classification Model. 
    - i have picked this way because in the lables are in classfication of either 0,1,2 so i have picked classfication model. i have also checked with other ways like language modeling but the accuracy is more in classfication model.
    a.    Naive Bayes Classifier
        i.    Naive Bayes is a classification algorithm based on the principle of Bayes' Theorem.
        •    works with any numbers of class
        •    Simple to implement

    b.    K-nearest Neighbour
        i.    It is a classification algorithm that operates on basic principal.
        •    Simple to implement
        •    works with any numbers of class
        •    easy to add more data

    c.    Support Vector Machine (SVM)
        i.    It analysed data and recognizer patterns through classification or regression techniques.
        •    SVM can handle non-linear data using the bit trick.
        •    SVM can be used to solve both classification and regression problems.

In [49]:
X_train = df[['ScoreFalse','scoreA','scoreB','scoreC']]
y_train = df['new_target']
y_test = df_train['new_target']
X_test = df_train[['ScoreFalse','scoreA','scoreB','scoreC']]

In [50]:

knn = KNeighborsClassifier(n_neighbors=10)

knn.fit(X_train,y_train)

y_pred = knn.predict(X_test)

print(accuracy_score(y_pred,y_test))

print(classification_report(y_pred,y_test))

0.346039603960396
              precision    recall  f1-score   support

           0       0.39      0.37      0.38       727
           1       0.35      0.35      0.35       678
           2       0.29      0.32      0.30       615

    accuracy                           0.35      2020
   macro avg       0.35      0.34      0.34      2020
weighted avg       0.35      0.35      0.35      2020



In [51]:
linear = SVC(gamma='auto')

linear.fit(X_train,y_train)

y_pred = linear.predict(X_test)

print(accuracy_score(y_pred,y_test))

print(classification_report(y_pred,y_test))

0.3311881188118812
              precision    recall  f1-score   support

           0       0.10      0.38      0.15       174
           1       0.09      0.34      0.15       186
           2       0.82      0.32      0.47      1660

    accuracy                           0.33      2020
   macro avg       0.34      0.35      0.26      2020
weighted avg       0.69      0.33      0.41      2020



In [3]:
# c.    Naive Bayes Classifier
        # i.    Naive Bayes is a classification algorithm based on the principle of Bayes' Theorem.
        # •    works with any numbers of class
        # •    Simple to implement

nb = MultinomialNB()

nb.fit(X_train,y_train)

y_pred = nb.predict(X_test)

print(accuracy_score(y_pred,y_test))

print(classification_report(y_pred,y_test))

NameError: name 'X_train' is not defined

# END